# Introdução ao Conjunto de Dados

Onde achar o dataset e informações: https://www.kaggle.com/code/pedrovinciusmeerholz/stellar-classification-with-mlp

**Sobre:**

A classificação estelar utiliza os dados espectrais das estrelas para categorizá-las em diferentes categorias. O sistema moderno de classificação estelar é conhecido como sistema de classificação Morgan-Keenan (MK). Ele utiliza o antigo sistema de classificação HR para categorizar as estrelas de acordo com sua cromaticidade e utiliza algarismos romanos para categorizar o tamanho da estrela. Neste conjunto de dados, estaremos utilizando a Magnitude Absoluta e o Índice de Cor B-V para identificar Gigantes e Anãs.

**Colunas do conjunto de dados:**

* **Vmag:** Magnitude aparente visual da estrela
* **Plx:** Distancia entre a estrela e a terra
* **e_Plx:** Erro do Plx (distancia entre a terra e a estrela). Se o erro for alto, temos que tratar esse problema, seja deletando a linha ou fazendo alguma outra alteração
* **B-V:** indice de cor B-V. Se perto de 0 ou negativo, é uma estrela quente. Se estiver perto de 2, é uma estrela fria. Outras estrelas estão em algum lugar no meio disso.
* **SpType:** tipo espectral
* **Amag:** magnitude absoluta da estrela
![image.png](attachment:37013e5e-2b0f-4089-9ff0-9bd83d3b9c54.png)
* **TargetClass:** Se a estrela é anã ou gigante. Se for anã, é 0. Se for gigante, é 1.



**Objetivo do projeto:**

Treinar um modelo de machine learning capaz de classificar, a partir dos dados das colunas das features (Vmag, Plx, e_Plx, B-V, SpType e Amag), a label (TargetClass) que aponta se a estrela é anã ou não.

Basicamente, o modelo irá aprender a partir dos exemplos que já temos no conjunto do dados os padrões contidos nas estrelas anãs e nas gigantes de forma que, quando uma nova observação para ele, ele será capaz de predizer a qual classe a estrela pertence mesmo que nós não tenhamos essa informação.

# Fazendo um Modelo Ruim

Nesse primeiro momento, queremos apenas entender os processo mais básicos para a criação de um modelo de machine learning simples. Com esse dataset, temos a intenção de criar um preditor capaz de predizer se uma certa estrela é anã ou gigante com base em algumas outras informações contidas no conjunto de dados.

Para realizar essa tarefa, vamos usar um algoritmo chamado KNN (k-nearest neighbors), que é um dos algoritmos mais básicos de machine learning. Mesmo sendo básico, tem um desempenho bom para a classificação muitas vezes.

Video Recomendado sobre KNN: https://www.youtube.com/watch?v=zvmbB3315Ko&t=518s&ab_channel=CANALSANDECO

Digo que é um modelo ruim pois não iremos fazer diversas etapas que aumentariam o desempenho. Vamos apenas fazer o mínimo necessário para o algoritmo funcionar.

## Entendendo o Dataset

In [4]:
# importação de bibliotecas
import pandas as pd

In [5]:
# lê o dataset
df = pd.read_csv("https://raw.githubusercontent.com/semcovici/Datasets/main/StellarClassification/Star39552_balanced.csv")

In [6]:
# imprime o dataset
df

,Vmag,Plx,e_Plx,B-V,SpType,Amag,TargetClass
0,10.00,31.66,6.19,1.213,K7V,22.502556,1
1,8.26,3.21,1.00,1.130,K0III,15.792525,0
2,8.27,12.75,1.06,0.596,F9V,18.797552,1
3,6.54,5.23,0.76,1.189,K1III,15.132508,0
4,8.52,0.96,0.72,0.173,B8V,13.431356,1
...,...,...,...,...,...,...,...
39547,5.83,0.17,0.52,0.474,B7Iab,6.982245,0
39548,7.05,18.12,0.92,0.424,F5V,18.340790,1
39549,9.21,3.89,1.46,0.227,A1IV,17.159748,1
39550,9.01,2.13,1.46,1.467,M5III,15.651898,0


## Preparação de Dados

Uma parte bastante importante no machine learning é a preparação de nossos dados. Temos realizar diversos tratamentos em nossos dados para melhorar o desempenho de nosso modelo. Também temos que preparar os nossos dados para que o algoritmo reconheça eles, muitas algoritmos não aceitam como entrada dados categoricos e muitos algoritmos não aceitam dados númericos. Então, temos que tranforma-los de alguma forma.

### Dados Nulos

Muitas vezes, vamos ter dados nulos ou faltantes no conjunto de dados. Basicamente, uma célula onde irá aparecer um NaN, ?, -1, etc indicando que aquele dados não existe no conjunto de dados.

Dessa forma, precisamos averiguar se o nosso dataset tem ou não esses dados faltantes. Para isso, utilizamos o método ".isnull()" que indica, através de True ou False, se nas células há um valor faltante.

In [7]:
df.isnull()

,Vmag,Plx,e_Plx,B-V,SpType,Amag,TargetClass
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...
39547,False,False,False,False,False,False,False
39548,False,False,False,False,False,False,False
39549,False,False,False,False,False,False,False
39550,False,False,False,False,False,False,False


Acima, não conseguimos ver o dataset inteiro e, mesmo que conseguissemos, seria bem desagradável ficar olhando célula por célula se há valores faltantes. Desse forma, como False é 0 e True é 1, se somarmos os valores das células após utilizar o '.isnull' vamos ter a quantidade de dados nulos no conjunto.

In [8]:
df.isnull().sum()

Vmag           0
Plx            0
e_Plx          0
B-V            0
SpType         0
Amag           0
TargetClass    0
dtype: int64

Não temos dados nulos. Se tivessemos, teriamos que tratar eles.

## Tratando as Variáreis Categoricas

O KNN é um algoritmo de machine learning que não funciona a partir de dados categoricos, justamente por ser um modelo baseado em distancias. No nosso caso, como vimos acima, temos a variável/coluna SpType que é categorica.

Então, o que fazemos? Temos algumas alternativas, a mais fácil é simplesmente tirar essa coluna e ignorar ela para a criação do modelo, porém poderiamos estar perdendo informação fazendo isso. Também poderiamos arrumar métodos de transformar essa variável categorica em numérica, o que faremos no futuro provavelmente. Por enquanto, vamos só excluí-la para prosseguir para a criação do modelo.

In [9]:
# tira a coluna
df.pop('SpType')
# o metodo pop tira a coluna do conjunto e retorna ela. Então, se colocassemos x = df.pop('SpType'), x seria apenas a coluna 'SpType'.
# Como só queremos tirar a coluna, só utilizamos o pop.
# Também é extramamente comum usar o método .drop('coluna', axis=1)

0          K7V
1        K0III
2          F9V
3        K1III
4          B8V
         ...  
39547    B7Iab
39548      F5V
39549     A1IV
39550    M5III
39551      F5V
Name: SpType, Length: 39552, dtype: object

In [10]:
df.head()

,Vmag,Plx,e_Plx,B-V,Amag,TargetClass
0,10.00,31.66,6.19,1.213,22.502556,1
1,8.26,3.21,1.00,1.130,15.792525,0
2,8.27,12.75,1.06,0.596,18.797552,1
3,6.54,5.23,0.76,1.189,15.132508,0
4,8.52,0.96,0.72,0.173,13.431356,1


Agora não temos mais a coluna.  

## Implementação do Modelo

In [11]:
# importando métodos do sklearn
from sklearn.neighbors import KNeighborsClassifier # knn
from sklearn.model_selection import train_test_split # separação de treino e teste
from sklearn.metrics import accuracy_score # para medir a acurácia

Para facilitar, vamos separar as features (X) e label (y):

In [12]:
y = df.pop('TargetClass')
X = df

In [13]:
print('Features:')
print(X)
print('\n\nLabel:')
print(y)

Features:
        Vmag    Plx  e_Plx    B-V       Amag
0      10.00  31.66   6.19  1.213  22.502556
1       8.26   3.21   1.00  1.130  15.792525
2       8.27  12.75   1.06  0.596  18.797552
3       6.54   5.23   0.76  1.189  15.132508
4       8.52   0.96   0.72  0.173  13.431356
...      ...    ...    ...    ...        ...
39547   5.83   0.17   0.52  0.474   6.982245
39548   7.05  18.12   0.92  0.424  18.340790
39549   9.21   3.89   1.46  0.227  17.159748
39550   9.01   2.13   1.46  1.467  15.651898
39551   9.12   3.82   0.79  0.480  17.030317

[39552 rows x 5 columns]


Label:
0        1
1        0
2        1
3        0
4        1
        ..
39547    0
39548    1
39549    1
39550    0
39551    1
Name: TargetClass, Length: 39552, dtype: int64


Também precismos separar o nosso conjunto entre treino e teste. Onde o treino usaremos para treinar o nosso modelo e, com o modelo pronto, utilizaremos o teste para avaliar se ele está adequado ou não.

E quantos dados usaremos para treino e quantos para teste?
Bem complicado utilizar alguma métrica objetiva para generalizar isso, exitem diversas formas de de fazer essa divisão. Como estamos apenas na introdução, vamos separar em 70% para treino e 30% para teste, que é uma divisão comum de se ver.

In [14]:
x_train,x_test ,y_train,y_test = train_test_split(X,y , test_size= 0.3)

Agora vamos começar a implementação do modelo. Para isso, precisamos

In [15]:
# cria o modelo
knn = KNeighborsClassifier(n_neighbors=5)

In [16]:
# treina o modelo
knn.fit(x_train, y_train)

In [ ]:
# Com o modelo treinado, prediz o y a partir do x_test
y_pred = knn.predict(x_test)

In [17]:
y_preds

array([1, 0, 1, ..., 0, 1, 0])

In [18]:
# mede a acurácia
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8692061351761335


Abaixo temos um exemplo prático. Coloque o valor das features que o modelo irá predizer se a estrela é anã ou não.

In [19]:
print('Crie um novo ponto: \n')

#input dos dados do ponto
new_Vmag = int(input("Vmag: "))
new_Plx = int(input("Plx: "))
new_e_Plx = int(input("e_Plx: "))
new_BV = int(input("B-V: "))
new_Amag = int(input("Amag: "))

#cria o novo ponto
new_point = [(new_Vmag, new_Plx, new_e_Plx, new_BV, new_Amag)]

# prediz a classe do ponto
new_pred = knn.predict(new_point)

if new_pred == 0: new_pred = 'anã'
else: new_pred = 'gigante'1é ' + new_pred)

Crie um novo ponto: 

Vmag: 1


KeyboardInterrupt: ignored